In [10]:
!pip uninstall -y bitsandbytes
!pip install -q "bitsandbytes>=0.43.0" "torch>=2.1" "transformers>=4.36" "accelerate>=0.25" "peft>=0.8" "datasets>=2.14"

Found existing installation: bitsandbytes 0.49.0
Uninstalling bitsandbytes-0.49.0:
  Successfully uninstalled bitsandbytes-0.49.0


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model, TaskType
from datasets import Dataset
import torch, json

model_name = "/content/drive/MyDrive/models/Qwen3-0.6B"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype=torch.float16,  # или torch.float32 для CPU
    device_map="auto",
    trust_remote_code=True,
)

print(f"✅ Модель {model_name} загружена на устройство: {model.device}")

✅ Модель /content/drive/MyDrive/models/Qwen3-0.6B загружена на устройство: cuda:0


In [10]:
def load_data(path):
    texts = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            if line.strip():
                item = json.loads(line)
                text = tokenizer.apply_chat_template(
                    item["messages"],
                    tokenize=False,
                    add_generation_prompt=False
                )
                texts.append({"text": text})
    return Dataset.from_list(texts)

dataset = load_data("/content/drive/MyDrive/data/reviews_train.jsonl")

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=512,
        padding="max_length"
    )

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=dataset.column_names
)

Map:   0%|          | 0/4046 [00:00<?, ? examples/s]

In [11]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 5,046,272 || all params: 601,096,192 || trainable%: 0.8395


In [12]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/qwen3-finetuned",
    per_device_train_batch_size=2,         # T4 выдержит 2–4
    gradient_accumulation_steps=4,
    max_steps=200,                          # 200 шагов ≈ 5–10 минут на T4
    learning_rate=2e-4,
    logging_steps=10,
    save_steps=100,
    fp16=True,                              # включаем для T4
    optim="paged_adamw_8bit",               # стабильнее на квантованных моделях
    report_to="none",
    save_safetensors=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.


Step,Training Loss
10,2.761100
20,2.217300
30,2.003900
40,1.981700
50,1.930400
60,1.898300
70,1.749900
80,1.745500
90,2.031600
100,1.864700


TrainOutput(global_step=200, training_loss=1.9029771089553833, metrics={'train_runtime': 419.5888, 'train_samples_per_second': 3.813, 'train_steps_per_second': 0.477, 'total_flos': 2189789075865600.0, 'train_loss': 1.9029771089553833, 'epoch': 0.3954522985664854})

In [13]:
model.save_pretrained("/content/drive/MyDrive/llm_finetune/models/Qwen3-0.6B-finetuned")
tokenizer.save_pretrained("/content/drive/MyDrive/llm_finetune/models/Qwen3-0.6B-finetuned")

('/content/drive/MyDrive/llm_finetune/models/Qwen3-0.6B-finetuned/tokenizer_config.json',
 '/content/drive/MyDrive/llm_finetune/models/Qwen3-0.6B-finetuned/special_tokens_map.json',
 '/content/drive/MyDrive/llm_finetune/models/Qwen3-0.6B-finetuned/chat_template.jinja',
 '/content/drive/MyDrive/llm_finetune/models/Qwen3-0.6B-finetuned/vocab.json',
 '/content/drive/MyDrive/llm_finetune/models/Qwen3-0.6B-finetuned/merges.txt',
 '/content/drive/MyDrive/llm_finetune/models/Qwen3-0.6B-finetuned/added_tokens.json',
 '/content/drive/MyDrive/llm_finetune/models/Qwen3-0.6B-finetuned/tokenizer.json')